# Impyte Documentation
This is a first practicle attempt to clarify the usage of `impyte`. It's a collection of easy applicable and reproducible examples that you could use to simplyify your data processing workflow.

## Importing and generating sample data
In order to show some of the features of the library, we'll be using Kaggle's HR data that can be found [here](https://www.kaggle.com/ludobenistant/hr-analytics).

In [1]:
# import library and data set
import impyte
import pandas as pd
from tools.data_prep import remove_random

In [2]:
data = pd.read_csv('data/hr_test.csv')

In [3]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Add random missing values
In order to show some of the pattern visualization and imputation methods, we need incomplete data sets. To achieve this, we'll be using a helper function that deletes values at random. 25 % of all values from each category will be deleted.

In [4]:
data = remove_random(data, .25)

In [5]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.48,3.0,169.0,3.0,0.0,0.0,0.0,support,medium
1,NaN,0.72,3.0,188.0,3.0,0.0,0.0,0.0,technical,low
2,NaN,0.51,4.0,274.0,2.0,0.0,0.0,0.0,support,medium
3,NaN,0.44,2.0,130.0,3.0,0.0,0.0,0.0,support,low
4,NaN,0.61,4.0,181.0,4.0,0.0,0.0,0.0,technical,low


In [6]:
data.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000
mean,0.613296,0.716067,3.801299,201.131145,3.500923,0.144137,0.237675,0.021060
std,0.248367,0.171141,1.233495,49.943865,1.464483,0.351240,0.425674,0.143589
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [7]:
len(data)

14999

In [8]:
reload(impyte)

<module 'impyte' from 'impyte.pyc'>

## Testing rudimentary features
Below is a first trial run of some of the features implemented by impyter and its helper classes.

## `NanChecker`
Functionality testing of `NanChecker` class.

In [9]:
nan_checker = impyte.NanChecker()

#### `NanChecker.is_nan(data, nan_vals=None, recursive=True)`
Detect missing values (NaN in numeric arrays, empty strings in string arrays). NaN detection feature also enables NaN-value ingestion as parameter values.

In [10]:
# [True, False, False]
print nan_checker.is_nan(["", 'None', 'NaN'])

# [True, True, True]
print nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

[True, False, False]
[True, True, True]


In [11]:
# Recursive nan detection
# [True, True, False, [False, True, True]]
print nan_checker.is_nan(["", None, 'NaN', ["List Value 1", '', None]])

[True, True, False, [False, True, True]]


In [12]:
# Values can be declared as nan-values
# [True, False, False, True]
nan_checker.is_nan(['NaN', 'Empty', 'None', 'N/A'], nan_vals=['NaN', 'N/A'])

[True, False, False, True]

## `Pattern`
Functionality testing of `Pattern` class. The `Pattern` class stores different patterns and data summaries regarding NaN values. 

#### `Pattern._get_discrete_and_continuous(tmpdata)`
Returns the column names of discrete and continuous variables. Column names are stored in lists for easy selection. Those lists are stored in one dictionary object. All continuous column names can be accessed through `['continuous']` all discrete variables through `['discrete']`.

In [13]:
reload(impyte)
pattern_log = impyte.Pattern()

In [14]:
pattern_log._get_discrete_and_continuous(data)

{'continuous': ['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years'],
 'discrete': ['sales', 'salary']}

#### `Pattern._compute_pattern(data, nan_values="", verbose=False)`
Function that checks for missing values and prints out a quick table of a summary of missing values. Includes pattern overview and counts of missing values by column. Computes pattern and returns its pattern structure plus the count of data points for each of these patterns. To get a nice digestible table add `["table"]` selector to output.

In [15]:
pattern_dict = pattern_log._compute_pattern(data)
indices, table = pattern_dict["indices"], pattern_dict["table"]

In [16]:
table

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [17]:
indices[0][:10] # first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

In [18]:
pattern_nr = 0
print "Pattern {} has {:,} rows.".format(pattern_nr, len(indices[0])) 

Pattern 0 has 11,259 rows.


#### `Pattern.get_pattern()`
Returns NaN-patterns based on primary computation or initiates new computation of NaN-patterns. Uses `_compute_patterns()` to visualize patterns if not yet computed. Otherwise returns stored information regarding patterns.

In [19]:
pattern_log.get_pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


#### `Pattern.row_nan_pattern(row)`
This is a core piece to determining which patterns are available. Other methods work with the output of `row_nan_pattern()` to determine unique pattern structures and count these before turning them into readable tables.

In [20]:
# (1, 1, 'NaN', 'NaN')
print pattern_log.row_nan_pattern(['Value 1', 'Value 2', '', None])

(1, 1, 'NaN', 'NaN')


#### `Pattern.get_pattern_indices(pattern_no)`

In [21]:
pattern_log.get_pattern_indices(0)[:10] # get first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

#### `Pattern.get_continuous()`
Returns list with names of all continuous variables.

In [22]:
pattern_log.get_continuous()

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years']

#### `Pattern.get_discrete()`
Returns list with names of all discrete variables.

In [23]:
pattern_log.get_discrete()

['sales', 'salary']

## `Impyter`

In [24]:
reload(impyte)
imputer = impyte.Impyter() # instantiate impyte class

### Load data into imputer

In [25]:
_ = imputer.load_data(data)

In [26]:
%timeit imputer.pattern()

The slowest run took 190445.57 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 5.01 µs per loop


In [27]:
%timeit imputer.get_pattern(4)

1000 loops, best of 3: 691 µs per loop


In [43]:
reload(impyte)
imputer = impyte.Impyter(data) # instantiate impyte class
nan_checker = impyte.NanChecker()
pattern_log = impyte.Pattern()
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [44]:
_ = imputer.drop_pattern(4, inplace=True)

In [45]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374
10,1,1,1,1,1,1,NaN,1,1,1,374


In [46]:
_ = imputer.load_data(data)

In [47]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [48]:
_ = imputer.drop_pattern(4)

In [49]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [52]:
for a in imputer.pattern_log.easy_access:
    print imputer.pattern_log.easy_access[a]

['sales']
['average_montly_hours']
['number_project']
['salary']
['promotion_last_5years']
['time_spend_company']
['satisfaction_level']
[]
['last_evaluation']
['Work_accident']
['left']


### Comparison of complete indices methods

In [35]:
%timeit len(imputer.pattern_log.get_complete_indices())

1000 loops, best of 3: 961 µs per loop


In [36]:
%timeit len(imputer.get_complete_old())

100 loops, best of 3: 6.42 ms per loop


In [37]:
_ = imputer.drop_pattern(7, inplace=True)

In [38]:
imputer.pattern_log.store_tuple_columns

{(1, 1, 1, 1, 1, 1, 1, 1, 1, 1): [],
 (1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN'): [9],
 (1, 1, 1, 1, 1, 1, 1, 1, 'NaN', 1): [8],
 (1, 1, 1, 1, 1, 1, 1, 'NaN', 1, 1): [7],
 (1, 1, 1, 1, 1, 1, 'NaN', 1, 1, 1): [6],
 (1, 1, 1, 1, 1, 'NaN', 1, 1, 1, 1): [5],
 (1, 1, 1, 1, 'NaN', 1, 1, 1, 1, 1): [4],
 (1, 1, 1, 'NaN', 1, 1, 1, 1, 1, 1): [3],
 (1, 1, 'NaN', 1, 1, 1, 1, 1, 1, 1): [2],
 (1, 'NaN', 1, 1, 1, 1, 1, 1, 1, 1): [1],
 ('NaN', 1, 1, 1, 1, 1, 1, 1, 1, 1): [0]}

In [39]:
data.iloc[:, imputer.pattern_log.store_tuple_columns[(1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN')]].columns

Index([u'salary'], dtype='object')

In [40]:
data.tail()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
14994,0.84,0.75,4.0,135.0,3.0,0.0,0.0,1.0,RandD,medium
14995,0.40,0.53,2.0,127.0,3.0,0.0,1.0,0.0,support,low
14996,0.84,0.41,6.0,255.0,6.0,1.0,0.0,0.0,sales,medium
14997,0.95,0.68,3.0,165.0,3.0,0.0,0.0,0.0,IT,medium
14998,0.97,0.61,4.0,262.0,3.0,1.0,0.0,1.0,marketing,medium


### Timing some of the functions
Below is a piece of information on efficiency of the functions. This doesn't reflect complexity of the functions, but it will show an runtime estimate for your local machine.

In [41]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

100000 loops, best of 3: 5.18 µs per loop
100000 loops, best of 3: 4.9 µs per loop


In [42]:
imputer.impute()

(['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years'],
 ['sales', 'salary'])